In [10]:
#import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import os
from collections import Counter

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [11]:
races = ['Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race']


In [56]:

combos = []
for voting_age in [0, 1]:
    for hispanic in [0, 1]:
        for race in range(63):
            combos.append([voting_age, hispanic, race])

#This method computes noisy population sizes for each (voting age Y/N)*(hispanic Y/N)*race combination using
# a geometric mechanism.
#-------------------------------------------------
#INPUT:
# dft: dataframe corresponding to a BLOCK
# ep: epsilon value for the county-wide mechanism
#-------------------------------------------------
#OUTPUT: 
# noisy_queries: numpy 3D array with dimensions (2, 2, 63)
#          the ith element is an integer representing the (noisy) population size for the category in combos[i]
def get_noisy_queries(dft,  ep):
    
    
    # here, we use epsilon = ep*0.12*0.5, as per Table 2.1 in https://www.nap.edu/read/25978/chapter/3#13
    geo = Geometric(epsilon=ep*0.12*0.5, sensitivity=1)
    
    countshape= 0
    queries = []
    noisy_queries = np.zeros(shape=(2, 2, 63))
    for voting_age in [0, 1]:
        for hispanic in [0, 1]:
            for race in range(63):
                queries.append([[voting_age, hispanic, race], 0])
    
    print(len(queries))
    
    
    # for i in range(len(queries)):
    #   category = queries[i][0]
    #   count = num entries in dft with those values of voting age, hispanic, race
    #   queries[i][1] = count
    #   noisy_queries[voting_age][hispanic][race] = geo.randomise(count) 
    
    for i in range(0,len(queries)):
        #age
        ag = queries[i][0][0]
        if queries[i][0][0] == 0:
            count = dft[dft['age']<18]
        else:
            count = dft[dft['age']>=18]
        #hispanic
        hispan = queries[i][0][1]
        count = count[count['hispanic']==hispan]
        #race
        ra = queries[i][0][2]
        count = count[count['race']==ra]
    
        countshape += count.shape[0]
    
        count_num = count.shape[0]
        queries[i][1] = count_num
        randomized =  geo.randomise(count_num)%len(queries)
        noisy_queries[ag][hispan][ra] = randomized
        

    
    
    #iterate through all hispanic/race
    # combos and add their population size to race_histogram
    
    race_histogram = np.zeros(shape=(2, 63))
    
    # race_histogram: numpy 2D array with dimension (2, 63)
    #          for row i,
    #          column 0 contains the population size of non-Hispanic individuals with race i
    #          column 1 contains the population size of Hispanic individuals with race i
    for i in range(2):
        for j in range(63):
            race_histogram[i][j] = noisy_queries[0][i][j] + noisy_queries[1][i][j]
    
    return race_histogram




#This method computes a (noisy) histogram of population size by race for the specified county, 
# summing over all blocks in the county the output of get_noisy_queries.
#--------------------------------------------------------------------------------------------
#INPUT: 
# county: dataframe containing county data  
# ep: epsilon value for the county-wide mechanism
#--------------------------------------------------------------------------------------------
#OUTPUT:


In [57]:
states = ['california', 'pennsylvania', 'newmexico', 'georgia', 'northdakota', 'hawaii', 'missouri', 'massachussets', 'vermont']
counties = ['alameda', 'armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']

states = ['california']
counties = ['alameda']

j=0
for county in counties:
    print(county)
    df_first = create_block_data('../homemade_data/'+county+'.csv')
    orig_df = df_first.copy()
    for index, row in orig_df.iterrows():
        d = row['race']
        orig_df.at[index, 'race'] = races.index(d)  
    racehist = get_noisy_queries(orig_df, 10)
    print(racehist)

alameda
252
[[180. 245.   4. 117. 251.   3.  11.   9.  18.   3.   9.   6.  15. 252.
    4. 248. 245. 245. 251.   4.   0.  10.   3. 255. 252. 248.  13.   0.
  498. 251. 252.   2. 254.   3. 249. 257.   3. 253.   3.   2. 254.   2.
    1. 498.   2. 499. 252. 251. 249. 498. 253.   4. 499.   9. 502.   1.
  252. 251. 252.   3.   4. 499. 250.]
 [195.  10.  14.   1. 251.  39.   3.   1.   9.   2.   0.   8.   2. 255.
  251.   4. 250.   8. 251.   1. 251. 251. 253. 502.   7.   2. 251. 500.
    2.   0. 252. 248. 250. 244.   3. 248. 251. 252.   5. 501. 250.   2.
    5.   0.   3. 248. 249. 502. 249. 251.   8.   1.   1. 499. 244.   1.
  248.   2.   1. 251.   0.   0. 248.]]
